In [5]:
import pandas as pd
import os
import matplotlib
import numpy as np
import json
import csv

In [6]:
file_path="C:\\Users\shail\Movies-ETL\wikipedia-movies.json"

In [7]:
#Using with and load load the file as list of dictionaries
with open(file_path,mode='r') as file:
    wiki_movies_raw=json.load(file)
#print(wiki_movies_raw[0]['Produced by'])
len(wiki_movies_raw)

7311

In [8]:
len(wiki_movies_raw)
wiki_movies__raw_df=pd.DataFrame(wiki_movies_raw)
#wiki_movies_df.head()
wiki_movies_df.columns.tolist()

NameError: name 'wiki_movies_df' is not defined

In [9]:
#loop through wiki_movies_raw and get only those movies who has a director and imdb link
wiki_movies=[movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and 'No. of episodes'not in movie
            ]
            

wiki_movies_df=pd.DataFrame(wiki_movies)
#wiki_movies_df.head(111)
#print(wiki_movies[111])

In [10]:

wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

6834    https://en.wikipedia.org/wiki/The_Insult_(film)
7058     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [11]:
def clean_movie(movie):
    movie = dict(movie)
    #List of alternative titles(languages)
    alternative_titles={}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alternative_titles[key]=movie[key]
            movie.pop(key)
    #print(movie['url'])
    #print(alternative_titles)
    if len(alternative_titles) >0:
        movie['alt_titles']= alternative_titles
        
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
            #print(movie['url'])
            #print("............")
            #print(f"{new_name},{movie[new_name]}")
            
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    
    
    
    return movie
    

In [12]:
clean_movies=[clean_movie(movie) for movie in wiki_movies]
#print(clean_movies[0])

In [13]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())


['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [14]:
#Columns are cleaned up..now clean up the rows
#Extract imdb id from movie url
wiki_movies_df['imdb_id']=wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id',inplace=True)
print(len(wiki_movies_df))

7076
7033


In [15]:
wiki_movies_df.columns

Index(['url', 'year', 'imdb_link', 'title', 'Based on', 'Starring',
       'Narrated by', 'Cinematography', 'Release date', 'Running time',
       'Country', 'Language', 'Budget', 'Box office', 'Director',
       'Distributor', 'Editor(s)', 'Composer(s)', 'Producer(s)',
       'Production company(s)', 'Writer(s)', 'Genre', 'Original language(s)',
       'Original network', 'Executive producer(s)', 'Production location(s)',
       'Picture format', 'Audio format', 'Voices of', 'Followed by',
       'Created by', 'Preceded by', 'Suggested by', 'alt_titles', 'Recorded',
       'Venue', 'Label', 'Animation by', 'Color process', 'imdb_id'],
      dtype='object')

In [ ]:
#Drop all the null values from all the columns
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df

In [16]:
len(wiki_movies_df)

7033

In [17]:
#Alternative for loop
for column in wiki_movies_df.columns:
    col=wiki_movies_df[column].isnull().sum()
    print(f"{column} : {col}")

url : 0
year : 0
imdb_link : 0
title : 1
Based on : 4852
Starring : 184
Narrated by : 6752
Cinematography : 691
Release date : 32
Running time : 139
Country : 236
Language : 244
Budget : 2295
Box office : 1548
Director : 0
Distributor : 357
Editor(s) : 548
Composer(s) : 518
Producer(s) : 202
Production company(s) : 1678
Writer(s) : 199
Genre : 6923
Original language(s) : 6875
Original network : 6908
Executive producer(s) : 6936
Production location(s) : 6986
Picture format : 6969
Audio format : 6972
Voices of : 7031
Followed by : 7024
Created by : 7023
Preceded by : 7023
Suggested by : 7032
alt_titles : 7012
Recorded : 7031
Venue : 7032
Label : 7031
Animation by : 7031
Color process : 7032
imdb_id : 0


In [18]:
wiki_movies_df.dtypes

url                       object
year                       int64
imdb_link                 object
title                     object
Based on                  object
Starring                  object
Narrated by               object
Cinematography            object
Release date              object
Running time              object
Country                   object
Language                  object
Budget                    object
Box office                object
Director                  object
Distributor               object
Editor(s)                 object
Composer(s)               object
Producer(s)               object
Production company(s)     object
Writer(s)                 object
Genre                     object
Original language(s)      object
Original network          object
Executive producer(s)     object
Production location(s)    object
Picture format            object
Audio format              object
Voices of                 object
Followed by               object
Created by

In [24]:
print(len(wiki_movies_df['Box office']))
box_office = wiki_movies_df['Box office'].dropna()
len(box_office)

7033


5485

In [25]:
def is_not_a_string(x):
    return type(x) != str

box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [27]:
#lambda version of is not a string function
#lambda x: type(x) != str

box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [ ]:
#load the kaggle data into df
movies_metadata="C:\\Users\shail\Movies-ETL\movies_metadata.csv"
movies_metadata_pd=pd.read_csv(movies_metadata,low_memory="False")

#movies_metadata_pd.head()

In [ ]:
ratings_path=os.path.join('ratings.csv')
ratings_df=pd.read_csv(ratings_path)
ratings_df.sample(n=7)